# Predictive Model

In [1]:
import pandas as pd
import numpy as np

#load data
house_rent = pd.read_csv('house_rent_2.csv', header = 0, na_values = [' ','NA','NAN','None','','N/A'])

In [2]:
#check data structure
print(house_rent.shape)
print(house_rent.count())

(329503, 15)
Unnamed: 0              329503
unique_id               329503
bathrooms               329503
bedrooms                329503
city                    329238
list_price              329503
latitude                329503
longitude               329503
property_type           329503
lot_sqft                329503
sqft                    329503
state                   329503
year_built              329503
zip                     329503
rentzestimate_amount    176420
dtype: int64


In [3]:
#check NA
# house_rent.loc[house_rent['year_built'].isnull()==True]

In [4]:
# #remove NA
# house_rent = house_rent.loc[house_rent['year_built'].isnull()== False]
#check data type
print(house_rent.dtypes)

Unnamed: 0                int64
unique_id                object
bathrooms               float64
bedrooms                  int64
city                     object
list_price              float64
latitude                float64
longitude               float64
property_type            object
lot_sqft                float64
sqft                    float64
state                    object
year_built                int64
zip                       int64
rentzestimate_amount    float64
dtype: object


In [5]:
#change zip code data type from float to string
house_rent.zip = house_rent.zip.astype(str)
# #change year_built data type from float to int
# house_rent.year_built = house_rent.year_built.astype(np.int64)
#create two variables: year_from_now and log_rent(the rent is right-skewed, log transformation on rent)
import datetime
house_rent['year_from_now'] = datetime.date.today().year - house_rent['year_built']
house_rent['log_rent'] = np.log(house_rent['rentzestimate_amount'])
# house_rent['log_lotsqft'] = np.log(house_rent['lot_sqft'])
# house_rent['log_listprice'] = np.log(house_rent['list_price'])

In [6]:
# house_rent.loc[house_rent['lot_sqft']== 0,'log_lotsqft'] = 0

In [7]:
#categorical variables: create dummies(state and property type)
#zip and city have too many levels, so I don't include them in the prediction
# state = pd.get_dummies(house_rent['state']).add_suffix('_state')
property_type = pd.get_dummies(house_rent['property_type']).add_suffix('_type')
house_rent_dummy = pd.concat([house_rent,property_type],axis = 1)

In [10]:
#divide the dataset into two datasets based on the existence of target variable
property_price_all = house_rent_dummy.loc[:,~house_rent_dummy.columns.isin(['Unnamed: 0','unique_id','city','property_type',
                                                                        'state','year_built','zip','rentzestimate_amount'
                                                       ])]
#rentzestimate_amount exits  
property_price_valid = property_price_all.loc[property_price_all['log_rent'].isnull()==False] 
#NA in rentzestimate_amount, to be predicted                                                                           
property_price_tobe = property_price_all.loc[property_price_all['log_rent'].isnull()==True] 
print(property_price_valid.shape)
print(property_price_valid.head(5))
print(property_price_tobe.shape)
print(property_price_tobe.head(5))

(176420, 20)
   bathrooms  bedrooms  list_price   latitude  longitude  lot_sqft    sqft  \
1        1.0         1     79500.0  30.587296 -98.401404    8712.0   957.0   
2        1.0         1     79500.0  30.587296 -98.401404    8712.0   957.0   
3        1.0         1    119000.0  30.587296 -98.401404    8712.0   957.0   
4        2.0         2    342850.0  30.742224 -98.444951       1.0  1592.0   
5        3.0         4    700000.0  30.546524 -97.797834       2.0  2808.0   

   year_from_now  log_rent  APT_type  COND_type  COOP_type  \
1             35  6.856462         1          0          0   
2             35  6.802395         1          0          0   
3             35  6.856462         1          0          0   
4             17  7.488294         0          0          0   
5             24  8.034955         0          0          0   

   Country Homes/Acreag_type  FARM_type  LAND_type  MULT_type  OTHER_type  \
1                          0          0          0          0       

In [12]:
# 10% sample to select the model first
property_price = property_price_valid.sample(frac = 0.1)
print(property_price.shape)
property_price.head()

(17642, 20)


,bathrooms,bedrooms,list_price,latitude,longitude,lot_sqft,sqft,year_from_now,log_rent,APT_type,COND_type,COOP_type,Country Homes/Acreag_type,FARM_type,LAND_type,MULT_type,OTHER_type,RENT_type,RESI_type,Timeshare_type
71363,3.0,4,749900.0,33.549054,-116.990608,160736.0,2359.0,44,8.247220,0,0,0,0,0,0,0,0,0,1,0
114862,0.0,3,299900.0,29.679956,-95.428505,6545.0,2012.0,24,7.244228,0,0,0,0,0,1,0,0,0,0,0
164534,3.0,2,1500000.0,40.801573,-73.647159,8712.0,3041.0,1,8.768419,1,0,0,0,0,0,0,0,0,0,0
111209,1.0,3,349900.0,29.798590,-95.408696,5000.0,988.0,92,7.377759,0,0,0,0,0,0,0,0,0,1,0
198819,2.0,3,279900.0,27.761389,-82.400963,6031.0,2180.0,11,7.408531,0,0,0,0,0,0,0,0,0,1,0


In [13]:
property_price.dtypes

bathrooms                    float64
bedrooms                       int64
list_price                   float64
latitude                     float64
longitude                    float64
lot_sqft                     float64
sqft                         float64
year_from_now                  int64
log_rent                     float64
APT_type                       uint8
COND_type                      uint8
COOP_type                      uint8
Country Homes/Acreag_type      uint8
FARM_type                      uint8
LAND_type                      uint8
MULT_type                      uint8
OTHER_type                     uint8
RENT_type                      uint8
RESI_type                      uint8
Timeshare_type                 uint8
dtype: object

In [14]:
#define features dataset(x) and target vector(y)
target = property_price.loc[:,property_price.columns.isin(['log_rent'])]
print('target:%s' %target.head())

features = property_price.loc[:,~property_price.columns.isin(['log_rent'])]
print('features:%s'%features.head())

target:        log_rent
71363   8.247220
114862  7.244228
164534  8.768419
111209  7.377759
198819  7.408531
features:        bathrooms  bedrooms  list_price   latitude   longitude  lot_sqft  \
71363         3.0         4    749900.0  33.549054 -116.990608  160736.0   
114862        0.0         3    299900.0  29.679956  -95.428505    6545.0   
164534        3.0         2   1500000.0  40.801573  -73.647159    8712.0   
111209        1.0         3    349900.0  29.798590  -95.408696    5000.0   
198819        2.0         3    279900.0  27.761389  -82.400963    6031.0   

          sqft  year_from_now  APT_type  COND_type  COOP_type  \
71363   2359.0             44         0          0          0   
114862  2012.0             24         0          0          0   
164534  3041.0              1         1          0          0   
111209   988.0             92         0          0          0   
198819  2180.0             11         0          0          0   

        Country Homes/Acreag_type 

In [15]:
from sklearn import linear_model,metrics,preprocessing,svm,cross_validation
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.cross_validation import train_test_split,cross_val_score,KFold
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import normalize

C:\Users\Hanyuan CHI\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
#standard normalization for each feature
standard_norm = preprocessing.StandardScaler()
feature_norm_part1 = standard_norm.fit_transform(features.iloc[:,0:8])
feature_norm = np.concatenate((feature_norm_part1,np.array(features.iloc[:,8:])),axis = 1)
target_array = np.squeeze(target.values)

In [17]:
#split training dataset and testing dataset
X_train, X_test, y_train, y_test = train_test_split(
    feature_norm, target_array, test_size=0.3, random_state=0)

In [18]:
#run regression models and calculate mse(mean squared error),rmse(root mean square error)and mae(mean absolute error)
#rmse on testing set shows the model generalization performance, which is used as model selection
for name,met in [
        ('neural network',MLPRegressor(hidden_layer_sizes=(30,30,30), activation='logistic', 
                                       solver='lbfgs', alpha=0.001)),
        ('regression tree',DecisionTreeRegressor(max_depth=20, min_samples_leaf= 5, min_samples_split= 5)),
        ('linear regression', LinearRegression()),
        ('ridge regression', Ridge(fit_intercept=True, alpha=0.3)),
        ('lasso regression', Lasso(fit_intercept=True, alpha=0.3)),
        ('elastic-net regularization', ElasticNet(fit_intercept=True, alpha=0.3)),
        ('KNN', KNeighborsRegressor(n_neighbors=2,metric = 'euclidean')),
        ('random forests regressor', RandomForestRegressor(n_estimators=100, criterion='mse', max_depth= 10, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features='auto', random_state=0)),
        ('extra trees regressor', ExtraTreesRegressor(n_estimators=100, criterion='mse', max_depth=10, 
                                                      min_samples_split=2, min_samples_leaf=1, 
                                                      min_weight_fraction_leaf=0.0, random_state=0)),
        ('gradient boosting regressor', GradientBoostingRegressor(loss='ls', alpha=0.95,
                                n_estimators=100, max_depth=10,
                                learning_rate=.1)),
        ('adaBoost regressor', AdaBoostRegressor(DecisionTreeRegressor(max_depth=10, min_samples_leaf= 5, min_samples_split= 5), 
                                                n_estimators=100, learning_rate=0.1, loss='linear', random_state=0)),
        ('bagging regressor',BaggingRegressor(DecisionTreeRegressor(max_depth=20, min_samples_leaf= 5, min_samples_split= 5),
                                              n_estimators=100,max_samples=1.0, max_features=1.0, random_state=0))
        ]:
    #cross validation to calculate rmse,mse,mae
    mse_cv = -cross_validation.cross_val_score(met, X_train, y_train, scoring='neg_mean_squared_error', cv=5,).mean()
    mae_cv = -cross_validation.cross_val_score(met, X_train, y_train, scoring='neg_mean_absolute_error', cv=5,).mean()
    rmse_cv = np.sqrt(abs(mse_cv))
    #generalizaton performance of the model on testing set
    met.fit(X_train, y_train)
    y_pred = met.predict(X_test)
    mse_test = metrics.mean_squared_error(y_test, y_pred)
    mae_test = metrics.mean_absolute_error(y_test, y_pred)
    rmse_test = np.sqrt(mse_test)
  
    print('Method: %s' %name)
    print('RMSE on testing: %.4f' %rmse_test)
    print('RMSE on CV: %.4f' %rmse_cv)
    print('MSE on testing: %.4f' %mse_test)
    print('MSE on CV: %.4f' %mse_cv)
    print('MAE on testing: %.4f' %mae_test)
    print('MAE on CV: %.4f' %mae_cv)
    print("\n")

Method: neural network
RMSE on testing: 0.2568
RMSE on CV: 0.2720
MSE on testing: 0.0660
MSE on CV: 0.0740
MAE on testing: 0.1536
MAE on CV: 0.1606


Method: regression tree
RMSE on testing: 0.2600
RMSE on CV: 0.2737
MSE on testing: 0.0676
MSE on CV: 0.0749
MAE on testing: 0.1548
MAE on CV: 0.1593


Method: linear regression
RMSE on testing: 0.4678
RMSE on CV: 0.4981
MSE on testing: 0.2188
MSE on CV: 0.2481
MAE on testing: 0.3405
MAE on CV: 0.3461


Method: ridge regression
RMSE on testing: 0.4678
RMSE on CV: 0.4981
MSE on testing: 0.2188
MSE on CV: 0.2481
MAE on testing: 0.3405
MAE on CV: 0.3461


Method: lasso regression
RMSE on testing: 0.6104
RMSE on CV: 0.6138
MSE on testing: 0.3726
MSE on CV: 0.3767
MAE on testing: 0.4547
MAE on CV: 0.4590


Method: elastic-net regularization
RMSE on testing: 0.5324
RMSE on CV: 0.5412
MSE on testing: 0.2834
MSE on CV: 0.2929
MAE on testing: 0.4005
MAE on CV: 0.4042


Method: KNN
RMSE on testing: 0.3508
RMSE on CV: 0.3682
MSE on testing: 0.1231
MS

With this sample, the performances of random forests, gradient boosting, adaBoost on Regression Tree and Bagging on Regression Tree are actually very similar in terms of RMSE testing set. Here we pick these four models out given their small RMSE on testing set, so now fit the model on the whole dataset to see its generalization performance.

In [19]:
#Random Forest

#define x and y
target = property_price_valid.loc[:,property_price_valid.columns.isin(['log_rent'])]

features = property_price_valid.loc[:,~property_price_valid.columns.isin(['log_rent'])]

#standard normalization for each feature
standard_norm = preprocessing.StandardScaler()
feature_norm_part1 = standard_norm.fit_transform(features.iloc[:,0:8])
feature_norm = np.concatenate((feature_norm_part1,np.array(features.iloc[:,8:])),axis = 1)
target_array = np.squeeze(target.values)

#split traing and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    feature_norm, target_array, test_size=0.3, random_state=0)

#fit random forests model on training set
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth= 10, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features='auto', random_state=0)

#use cross-validation to calculate rmse,mse,mae      
mse_cv = -cross_validation.cross_val_score(RF, X_train, y_train, scoring='neg_mean_squared_error', cv=5,).mean()
mae_cv = -cross_validation.cross_val_score(RF, X_train, y_train, scoring='neg_mean_absolute_error', cv=5,).mean()
rmse_cv = np.sqrt(abs(mse_cv))

#generalization performance of the model
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
mse_test = metrics.mean_squared_error(y_test, y_pred)
mae_test = metrics.mean_absolute_error(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
  
print('RMSE on testing: %.4f' %rmse_test)
print('RMSE on CV: %.4f' %rmse_cv)
print('MSE on testing: %.4f' %mse_test)
print('MSE on CV: %.4f' %mse_cv)
print('MAE on testing: %.4f' %mae_test)
print('MAE on CV: %.4f' %mae_cv)


RMSE on testing: 0.2154
RMSE on CV: 0.2126
MSE on testing: 0.0464
MSE on CV: 0.0452
MAE on testing: 0.1284
MAE on CV: 0.1271


In [20]:
#fit gradient boosting model on training set
GB = GradientBoostingRegressor(loss='ls', alpha=0.95,n_estimators=100, max_depth=10,learning_rate=.1)

#use cross-validation to calculate rmse,mse,mae      
mse_cv = -cross_validation.cross_val_score(GB, X_train, y_train, scoring='neg_mean_squared_error', cv=5,).mean()
mae_cv = -cross_validation.cross_val_score(GB, X_train, y_train, scoring='neg_mean_absolute_error', cv=5,).mean()
rmse_cv = np.sqrt(abs(mse_cv))

#generalization performance of the model
GB.fit(X_train, y_train)
y_pred = GB.predict(X_test)
mse_test = metrics.mean_squared_error(y_test, y_pred)
mae_test = metrics.mean_absolute_error(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
  
print('RMSE on testing: %.4f' %rmse_test)
print('RMSE on CV: %.4f' %rmse_cv)
print('MSE on testing: %.4f' %mse_test)
print('MSE on CV: %.4f' %mse_cv)
print('MAE on testing: %.4f' %mae_test)
print('MAE on CV: %.4f' %mae_cv)

RMSE on testing: 0.1841
RMSE on CV: 0.1846
MSE on testing: 0.0339
MSE on CV: 0.0341
MAE on testing: 0.1010
MAE on CV: 0.1010


In [21]:
Ada = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10, min_samples_leaf= 5, min_samples_split= 5), 
                                                n_estimators=100, learning_rate=0.1, loss='linear', random_state=0)
#use cross-validation to calculate rmse,mse,mae      
mse_cv = -cross_validation.cross_val_score(Ada, X_train, y_train, scoring='neg_mean_squared_error', cv=5,).mean()
mae_cv = -cross_validation.cross_val_score(Ada, X_train, y_train, scoring='neg_mean_absolute_error', cv=5,).mean()
rmse_cv = np.sqrt(abs(mse_cv))

#generalization performance of the model
Ada.fit(X_train, y_train)
y_pred = Ada.predict(X_test)
mse_test = metrics.mean_squared_error(y_test, y_pred)
mae_test = metrics.mean_absolute_error(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
  
print('RMSE on testing: %.4f' %rmse_test)
print('RMSE on CV: %.4f' %rmse_cv)
print('MSE on testing: %.4f' %mse_test)
print('MSE on CV: %.4f' %mse_cv)
print('MAE on testing: %.4f' %mae_test)
print('MAE on CV: %.4f' %mae_cv)

RMSE on testing: 0.2235
RMSE on CV: 0.2176
MSE on testing: 0.0500
MSE on CV: 0.0474
MAE on testing: 0.1418
MAE on CV: 0.1386


In [22]:
Bag = BaggingRegressor(DecisionTreeRegressor(max_depth=20, min_samples_leaf= 5, min_samples_split= 5),
                                              n_estimators=100,max_samples=1.0, max_features=1.0, random_state=0)
#use cross-validation to calculate rmse,mse,mae      
mse_cv = -cross_validation.cross_val_score(Bag, X_train, y_train, scoring='neg_mean_squared_error', cv=5,).mean()
mae_cv = -cross_validation.cross_val_score(Bag, X_train, y_train, scoring='neg_mean_absolute_error', cv=5,).mean()
rmse_cv = np.sqrt(abs(mse_cv))

#generalization performance of the model
Bag.fit(X_train, y_train)
y_pred = Bag.predict(X_test)
mse_test = metrics.mean_squared_error(y_test, y_pred)
mae_test = metrics.mean_absolute_error(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
  
print('RMSE on testing: %.4f' %rmse_test)
print('RMSE on CV: %.4f' %rmse_cv)
print('MSE on testing: %.4f' %mse_test)
print('MSE on CV: %.4f' %mse_cv)
print('MAE on testing: %.4f' %mae_test)
print('MAE on CV: %.4f' %mae_cv)

RMSE on testing: 0.1936
RMSE on CV: 0.1924
MSE on testing: 0.0375
MSE on CV: 0.0370
MAE on testing: 0.1042
MAE on CV: 0.1041


Gradient Boosting is best when working on the whole dataset. Since mean absolute error on testing set of gradient boosting model is 0.1010, which means log(rent_prediction)-log(rent_true) = +/-0.1010, after transformation, rent_prediction/rent_true = $e^{+/-0.1042}$ = 1.106/0.904, so the predicted rent is either 10.6% more than the actual rent or 9.6% less than the actual rent.

In [23]:
#Prediction for those records without rentzestimate_amount:property_price_tobe. First transform features of that dataset
X_tobe= property_price_tobe.loc[:,~property_price_tobe.columns.isin(['log_rent'])]
print('features:%s'%X_tobe.head())

standard_norm = preprocessing.StandardScaler()
X_tobe_norm_part1 = standard_norm.fit_transform(X_tobe.iloc[:,0:8])
X_tobe_norm = np.concatenate((X_tobe_norm_part1,np.array(X_tobe.iloc[:,8:])),axis = 1)

features:    bathrooms  bedrooms  list_price   latitude  longitude  lot_sqft    sqft  \
0         2.0         2    249900.0  30.889162 -98.465741       1.0  2324.0   
6         3.0         4    550000.0  30.129987 -97.779938       4.0  2500.0   
10        1.0         3     38528.0  33.208160 -88.116701       0.0  1123.0   
16        2.0         4     29500.0  42.433614 -82.996288    8712.0  1431.0   
19        1.0         2     25000.0  33.208160 -88.116701    8712.0  1165.0   

    year_from_now  APT_type  COND_type  COOP_type  Country Homes/Acreag_type  \
0              67         0          0          0                          0   
6               2         0          0          0                          0   
10             57         0          0          0                          0   
16             78         0          0          0                          0   
19             87         0          0          0                          0   

    FARM_type  LAND_type  MULT_type

In [24]:
# Use Gradient Boosting modelt to generate predicitons for three datasets.
y_pred_train = GB.predict(X_train)
y_pred_test = GB.predict(X_test)
y_pred_tobe = GB.predict(X_tobe_norm)

train_pred = pd.concat([pd.DataFrame(X_train),pd.DataFrame(y_train),pd.DataFrame(y_pred_train)],axis = 1)
test_pred = pd.concat([pd.DataFrame(X_test),pd.DataFrame(y_test),pd.DataFrame(y_pred_test)],axis = 1)
tobe_pred = pd.concat([pd.DataFrame(X_tobe_norm),pd.DataFrame(y_pred_tobe)],axis = 1)

pd.DataFrame(train_pred).to_csv('train_prediction_sk.csv')
pd.DataFrame(test_pred).to_csv('test_prediction_sk.csv')
pd.DataFrame(tobe_pred).to_csv('tobe_prediction_sk.csv')

In [26]:
#give column names
X_train = pd.DataFrame(X_train)
X_train.columns = features.columns.values

In [27]:
pd.DataFrame(X_train.columns)

,0
0,bathrooms
1,bedrooms
2,list_price
3,latitude
4,longitude
5,lot_sqft
6,sqft
7,year_from_now
8,APT_type
9,COND_type


In [30]:
np.argsort(GB.feature_importances_)

array([18, 10, 15, 12, 16, 11, 13,  9, 14,  8, 17,  1,  0,  5,  7,  6,  3,
        4,  2], dtype=int64)

In [31]:
#check importance of features
importances = GB.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d.  %15s  (%f)" % (f + 1, X_train.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1.       list_price  (0.348840)
2.        longitude  (0.184163)
3.         latitude  (0.165592)
4.             sqft  (0.089486)
5.    year_from_now  (0.066646)
6.         lot_sqft  (0.064469)
7.        bathrooms  (0.035840)
8.         bedrooms  (0.022900)
9.        RESI_type  (0.005252)
10.         APT_type  (0.004918)
11.        MULT_type  (0.003021)
12.        COND_type  (0.002650)
13.        LAND_type  (0.002624)
14.  Country Homes/Acreag_type  (0.002589)
15.        RENT_type  (0.000707)
16.        FARM_type  (0.000196)
17.       OTHER_type  (0.000106)
18.        COOP_type  (0.000000)
19.   Timeshare_type  (0.000000)
